In [1]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import time
import ipywidgets.widgets as widgets
from cvlib.object_detection import YOLO
import threading

In [6]:
yolo = YOLO('yolov4-tiny-custom_best.weights', 'yolov4-tiny-test.cfg', 'obj.names')

[INFO] Initializing YOLO ..


In [41]:
def camera(capture):
    start_time = time.time()
    frame_count = 0
    fps_total = 0
    
    global camera_stop, HSV_learning
    while capture.isOpened() and not camera_stop:
        _, img = capture.read()

        bbox, label, conf = yolo.detect_objects(img)
        yolo.draw_bbox(img, bbox, label, conf, write_conf=True)
        
        cur_time = time.time()
        elapsed = cur_time - start_time
        start_time = cur_time
        fps = 1/elapsed
        frame_count += 1
        fps_total += fps
        avg_fps = fps_total/frame_count
        fps_txt = "FPS: {:.2f}, Avg FPS: {:.2f}".format(fps, avg_fps)
        cv.putText(img, fps_txt, (30, 430), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (0,0,255), 1)
        
        image_widget.value = cv.imencode('.jpg', img)[1].tobytes()

    capture.release()
    cv.destroyAllWindows()

In [44]:
capture = cv.VideoCapture(0)
camera_stop = 0
image_widget = widgets.Image(format='jpeg', width=640, height=480)
display(image_widget)
threading.Thread(target=camera, args=(capture,)).start()

Image(value=b'', format='jpeg', height='480', width='640')

In [45]:
camera_stop = 1